# Hi kagglers 🙋🏻‍♂️ and Welcome to this new competition!

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import random
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
import collections
from nltk.corpus import stopwords
import string
from sklearn import preprocessing
from sklearn.manifold import TSNE
import seaborn as sns
from nltk.stem.porter import PorterStemmer
from sklearn import svm
from nltk.tokenize import word_tokenize
from time import time
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [ ]:
train = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
test = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
sub = pd.read_csv('/kaggle/input/commonlitreadabilityprize/sample_submission.csv')

# Let's start with EDA

In [ ]:
print(train.shape,test.shape)

In [ ]:
train.head(3)

In [ ]:
test.head(3)

In [ ]:
#Check if there'is null values
train.isnull().sum()

In [ ]:
# Target distibution
plt.figure(figsize=(10,4))
#plt.xlim(train['target'].min(),train['target'].max())
sns.kdeplot(train['target'],shade=True)
plt.show()

In [ ]:
# Numbers of word for each sapmle in train & test data
train['text_length'] = train.excerpt.apply(lambda x: len(x.split()))
test['text_length'] = test.excerpt.apply(lambda x: len(x.split()))

In [ ]:
train['text_length'].describe()

In [ ]:
test['text_length'].describe()

In [ ]:
def plot_word_count(df, data_name):
  sns.distplot(df['text_length'].values)
  plt.title(f'Sequence char count: {data_name}')
  plt.grid(True)

In [ ]:
#ig = plt.figure(figsize=(16,6))
#plt.hist(train["text_length"], bins = 30)
#plt.show()
plt.subplot(1, 2, 1)
plot_word_count(train, 'Train')

plt.subplot(1, 2, 2)
plot_word_count(test, 'Test')

plt.subplots_adjust(right=3.0)
plt.show()

In [ ]:
# collecting all words in single list
list_= []
for i in train.excerpt:
    list_ += i
list_= ''.join(list_)
allWords=list_.split()
vocabulary= set(allWords)

In [ ]:
len(vocabulary)

 * We have 56087 different words in our train data

In [ ]:
def create_corpus(df,column_name):
    corpus=[]
    
    for x in df[column_name].str.split():
        for i in x:
            corpus.append(i)
    return corpus

In [ ]:
#most frequent 30 words  
allWords=create_corpus(train,'excerpt')
vocabulary= set(allWords)
vocabulary_list= list(vocabulary)

plt.figure(figsize=(14,10))
counter=collections.Counter(allWords)
most=counter.most_common()
x=[]
y=[]
for word,count in most[:30]:
  x.append(word)
  y.append(count)
sns.barplot(x=y,y=x)

# Data Cleaning

## 1-Removing Punctuations

In [ ]:
#List of punctuations and we will remove them from our corpus
string.punctuation

In [ ]:
#for  example
text='hey # how are !you doing ?'
"".join([char for char in text if char not in string.punctuation])

## 2-Removing Numbers

In [ ]:
#for example 
text='hey 4 look 333 at me0'
re.sub('[0-9]', '', text)

## 3-Removing Stopwords

In [ ]:
#list of stopwords
stopwords.words('english')

In [ ]:
#for example
text='hey this is me and I am here to help you  '
tokens = word_tokenize(text)
tokens=[word for word in tokens if word not in stopwords.words('english')]
' '.join(tokens)

## Now let's Build a function that clean our data

* I just added lower function in order to lowercase all words and stemming

In [ ]:
pstem = PorterStemmer()
def clean_text(text):
    text= text.lower()
    text= re.sub('[0-9]', '', text)
    text  = "".join([char for char in text if char not in string.punctuation])
    tokens = word_tokenize(text)
    tokens=[pstem.stem(word) for word in tokens]
    tokens=[word for word in tokens if word not in stopwords.words('english')]
    text = ' '.join(tokens)
    return text

In [ ]:
clean_text("hey I am here # ! looks 4 GOOD can't see you!")

In [ ]:
train["clean"]=train["excerpt"].apply(clean_text)
test["clean"]=test["excerpt"].apply(clean_text)

In [ ]:
#Let's see the effect of cleaning
train[["excerpt","clean"]].head(4)

In [ ]:
# collecting all words in single list
list_= []
for i in train.clean:
    list_ += i
list_= ''.join(list_)
allWords=list_.split()
vocabulary= set(allWords)
len(vocabulary)

* we reduced our data from 56087 unique words to 22073

In [ ]:
#most frequent 30 words after the cleaning of text 
allWords=create_corpus(train,'clean')
vocabulary= set(allWords)
vocabulary_list= list(vocabulary)

plt.figure(figsize=(14,10))
counter=collections.Counter(allWords)
most=counter.most_common()
x=[]
y=[]
for word,count in most[:30]:
  x.append(word)
  y.append(count)
sns.barplot(x=y,y=x)

In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, norm='l2',ngram_range=(1,1))
features = tfidf.fit_transform(train.clean).toarray()
features.shape

In [ ]:
features_test = tfidf.transform(test.clean).toarray()

# Let's use LightGbm Regressor

In [ ]:
params = {'metric': 'rmse','random_state': 48,'n_estimators': 20000,'reg_alpha': 0.0010819683712588644,
          'reg_lambda': 0.004760428916800031, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.01,
          'max_depth': 100, 'num_leaves': 39, 'min_child_samples': 12, 'cat_smooth': 67}

In [ ]:
preds = np.zeros(test.shape[0])
kf = KFold(n_splits=5,random_state=48,shuffle=True)
rmse=[]  # list contains rmse for each fold
n=0
for trn_idx, test_idx in kf.split(features,train['target']):
    X_tr,X_val=features[trn_idx],features[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]
    model = LGBMRegressor(**params)
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    preds+=model.predict(features_test)/kf.n_splits
    rmse.append(mean_squared_error(y_val, model.predict(X_val), squared=False))
    print(n+1,rmse[n])
    n+=1

In [ ]:
np.mean(rmse)

In [ ]:
# Prediction distibution
plt.figure(figsize=(10,4))
sns.kdeplot(preds,shade=True)
plt.show()

# Submission

In [ ]:
sub['target']=preds
sub.to_csv('submission.csv', index=False)

In [ ]:
sub

# Please If you find this kernel helpful, upvote it to help others see it 😊